In [7]:
#2019 number of cars in uk or lon (thousands) [VEH0204]
uk_car_2019=32884.30
lon_car_2019=2661

# adoption rate of fuel type in GB yearly from 2001/2015 to 2020 [VEH0253]
# number of newly registered vehicles of fuel type / total new registrations in that year
# data sourced from DfT's VEH0253
adop_car_p=[82.1,76.2,72.3,67.3,63.2,61.5,59.3,56.0,57.8,53.1,48.5,48.1,49.0,48.1,49.0,49.3,53.5,62.3,65.6,62.5]
adop_car_d=[17.8,23.7,27.5,32.6,36.6,38.1,40.0,43.2,41.4,45.7,50.3,50.5,49.5,49.8,48.2,47.4,41.8,31.4,26.3,18.9]
adop_car_h=[0]*3+[0.1,0.2,0.4,0.7,0.7,0.7,1.1,1.2,1.2,1.3,1.5,1.7,1.9,2.8,3.7,4.9,9.4]
adop_car_ph=[0]*13+[0.3,0.7,1.0,1.3,1.8,1.5,3.3]
adop_car_b=[0]*10+[0.1,0.1,0.1,0.3,0.4,0.4,0.5,0.7,1.6,5.8]

#2019 ages of cars in % of cars at given age starting from 0 to 30+ (i.e. 2019 to <1989)
car_per_age=[6.701865813,6.865554769,7.316778939,7.803457088,7.685096727,7.201434605,6.537954259,5.55890771,5.143852754,5.264539452,5.021894934,
     4.926460392,5.162837486,4.390143996,3.717893513,3.07826344,2.417367913,1.756067938,1.103872606,0.678520949,0.452388859,0.305495142,
     0.21615469,0.146699095,0.111399597,0.07995604,0.063239866,0.052973575,0.060591188,0.05324118,0.087805982]
#swap order of cars (oldest first) and put in decimal form
car_age=list(car_per_age[-i]/100 for i in range(0,31))

#average fuel consumption in litres per 100km for new cars registered from 1997 to 2019 (ENV0103)
fuel_car_p=[8.3,8.3,8.1,8.0,7.9,7.8,7.7,7.6,7.5,7.4,7.2,7.0,6.5,6.3,6.1,5.8,5.6,5.5,5.4,5.4,5.5,5.6,5.7]
fuel_car_d=[7.0,6.9,6.6,6.3,6.2,6.1,6.2,6.2,6.2,6.3,6.2,5.9,5.7,5.5,5.2,5.0,4.9,4.7,4.6,4.5,4.6,4.9,5.1]

#kgC02 per litre of petrol and diesel [from DEFRA 2020 conversion factors]
petrol=2.30176
diesel=2.65242

#electricity emission factors kgco2/kwh (grid including losses) from 1989 to 2020 [source 2019 ghg conversion factors]
elec_emissions=[0.760,0.741,0.715,0.671,0.602,0.593,0.562,0.532,0.489,0.498,0.463,0.487,0.511,0.495,0.519,0.515,0.500,0.520,0.530,0.521,0.488,0.493,0.480,0.533,0.496,0.446,0.381,0.305,0.275,0.281,0.254,0.231]

#km driven per year by cars & taxis in GB and london from 1994 to 2019 (billion km) [TRA0201/0206]
km_gb=[345.0,351.1,359.9,365.8,370.6,377.4,376.0,381.2,390.6,390.0,394.2,392.7,397.4,397.9,395.0,394.0,389.2,393.2,395.1,396.9,408.0,415.4,424.7,432.9,438.3,447.8]
km_lon=[25.9,25.8,26,26.1,26.2,26.8,26.5,26.3,26.3,25.7,25.4,25.1,24.8,24.4,23.9,23.9,24.7,24.5,24.6,24.7,25.4,25.6,25.7,26.3,27.3,28.1]


In [8]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

class Mathematics:
    """
    Creates straight and polynomial fit functions
    """
    def poly_fit(years,data,power):
        """
        Generate polynomial fit of given data.
        Inputs: x values (usually years), data, power (1 for linear, 2 for quadratic)
        """
        coeff = np.polyfit(years,data,power,cov=False)
        function=sp.poly1d(coeff)
        return(function)
    
    def straight_fit(x1,y1,x2,y2,f):
        """
        Straight fit between two points
        """
        m = (y2 - y1) / (x2 - x1)
        c = (y2 - (m * x2))
        return [i*m+c for i in f]

class Vehicle:
    """
    Creates vehicle object with emission properties
    """
    def __init__(self, body_type, fuel_type, age, mass, china, elec):
        self.body_type = body_type
        self.fuel_type = fuel_type
        self.age = age 
        self.m=mass
        self.c=china
        self.e=elec
        #self.y=y
        #self.emissions = emissions
       
    def emissions(self,y):
        """
        Specifies the co2 emissions intensity of every fuel type (g/km)
        """
        emissions_value=0
        phev_electric_emissions=0
        phev_tailpipe_emissions=0
        #if BEV
        if self.fuel_type==4:
            emissions_value=np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[self.e][y-2010])
            #emissions_value=np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[0][y-1989])
        #if petrol
        if self.fuel_type==1:
            #[petrol]=kgCO2/l, [fuel consumption]=l/100km, so *10 gives [emissions]=g/km
            emissions_value=petrol*10*np.array(Fuel_Consumption(self.m).petrol()[self.age-1989])
        #if diesel
        if self.fuel_type==0:
            emissions_value=diesel*10*np.array(Fuel_Consumption(self.m).diesel()[self.age-1989])
        #if hybrid
        if self.fuel_type==2:
            #emissions_value=petrol*10*np.array(Fuel_Consumption(self.m).hybrid()[1][self.age-1989])
            
            emissions_value=0.5*(petrol*10*np.array(Fuel_Consumption(self.m).petrol()[self.age-1989])
                                 +np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[self.e][y-2010]))
            #emissions_value=0.5*(petrol*10*np.array(Fuel_Consumption(self.m).petrol()[self.age-1989])
            #                     +np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[0][y-1989]))
            #real-world PHEV utility factor of 39% as found by the ICCT
            phev_electric_emissions=0.39*np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[self.e][y-2010])
            #phev_electric_emissions=0.5*np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[0][y-1989])
            phev_tailpipe_emissions=0.61*petrol*10*np.array(Fuel_Consumption(self.m).petrol()[self.age-1989])
        #if retrofitted ICEV
        if self.fuel_type==3:
            emissions_value=np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[self.e][y-2010])
            #emissions_value=np.array(Fuel_Consumption(self.m).bev()[self.age-1989])*10*np.array(Fuel_Consumption(self.m).electricity()[0][y-1989])
        
        
        return emissions_value,phev_electric_emissions,phev_tailpipe_emissions
    
    def prod_emissions(self,y):
        """
        Specifies the production emissions of each type of vehicle (kgco2)
        Consists of emissions from manufacturing + end-of-life emissions + emissions change with mass 
        + emissions of road construction
        """
        #ICE from eu, EVs from eu or china)
        emissions_value=0
        #if BEV
        if self.fuel_type==4:
            #china
            if self.c==1:
                emissions_value=18330+(self.m-1400)*4+4944
            #eu
            elif self.c==0:
                #emissions_value=12080+(self.m-1400)*4
                #half of manufacturing energy gets decarbonised with grid
                emissions_value=4349+(self.m-1400)*4+(48450/3.6)*Fuel_Consumption(self.m).electricity()[self.e][y-2010]+4944       
        #if petrol
        if self.fuel_type==1:
            #emissions_value=11260+(self.m-1400)*4
            emissions_value=2117+(self.m-1400)*4+(34850/3.6)*Fuel_Consumption(self.m).electricity()[self.e][y-2010]+4944
        #if diesel
        if self.fuel_type==0:
            #emissions_value=12115+(self.m-1400)*4
            emissions_value=2117+(self.m-1400)*4+(34850/3.6)*Fuel_Consumption(self.m).electricity()[self.e][y-2010]+4944
        #if hybrid
        if self.fuel_type==2:
            #china
            if self.c==1:
                emissions_value=18490+(self.m-1400)*4+4944
            #eu
            elif self.c==0:
                #emissions_value=12490+(self.m-1400)*4
                emissions_value=2769+(self.m-1400)*4+(42650/3.6)*Fuel_Consumption(self.m).electricity()[self.e][y-2010]+4944
        #if retrofit
        if self.fuel_type==3:
            #china
            if self.c==1:
                emissions_value=8043+4944
            #eu
            elif self.c==0:
                #emissions_value=4575
                emissions_value=2500+(25000/3.6)*Fuel_Consumption(self.m).electricity()[self.e][y-2010]+4944
        return emissions_value
    
    def prod_energy(self):
        """
        Specifies the production energy of each type of vehicle
        """
        #in MJ per car
        #manufacturing+endoflife energy + energy change with mass + energy of road construction
        energy_value=0
        #if BEV
        if self.fuel_type==4:
            energy_value=96900+((self.m-1400)/1400)*0.75*96900+72564
        #if petrol
        if self.fuel_type==1:
            energy_value=69700+((self.m-1400)/1400)*0.8*69700+72564
        #if diesel
        if self.fuel_type==0:
            energy_value=69700+((self.m-1400)/1400)*0.8*69700+72564
        #if hybrid
        if self.fuel_type==2:
            energy_value=85300+((self.m-1400)/1400)*0.75*85300+72564
        #if retrofit
        if self.fuel_type==3:
            energy_value=50000
        return energy_value
    
class ModalShare:
    """
    Finds emissions and energy intensity of non-car modes
    """
    def co2intensity(elec):
        """
        CO2 intensity of non-car modes
        Can adjust modal shares and emissions intensities (e.g. due to changing occupancies) to find lower non-car mode emissions
        """
        #non car modal share of london
        modal_walk=[0.39]*31
        modal_cycle=[0.04]*31
        modal_bus=[0.22]*31
        modal_rail=[0.35]*31
        modal_ferry=[0]*31
        
        #kgco2-eq per passenger kilometer transport (pkt)
        #values based on sustainability paper sustainability-650843-SI
        co2walk=[0.00025]*31
        co2cycle=[0.0094]*31
        #at bus occupancy of 20
        #keeping emissions from manufacture and infrastructure relatively fixed 
        #emissions from energy use per km is straight line between 2020 co2 value and 2050 100% electricity value
        co2bus=np.array([0.023]*31)+Mathematics.straight_fit(2019,0.088,2050, \
                                                             0.319*Fuel_Consumption(1400).electricity()[elec][-1],range(2020,2051))
        #use electricity values for 2020 and 2050
        co2rail=np.array([0.013]*31)+Mathematics.straight_fit(2019,0.15*Fuel_Consumption(1400).electricity()[elec][10],2050, \
                                                             0.15*Fuel_Consumption(1400).electricity()[elec][-1],range(2020,2051))
        
        avg=np.array(modal_walk)*np.array(co2walk)+np.array(modal_cycle)*np.array(co2cycle) \
            +np.array(modal_bus)*np.array(co2bus)+np.array(modal_rail)*np.array(co2rail)
        
        return avg
    
    def energy():
        """
        Energy intensity of non-car modes
        """
        #non car modal share of london
        modal_walk=[0.39]*31
        modal_cycle=[0.04]*31
        modal_bus=[0.22]*31
        modal_rail=[0.35]*31
        modal_ferry=[0]*31
        
        #MJ per passenger kilometer transport (pkt)
        #values based on sustainability paper [sustainability-650843-SI]
        walk=[0.007]*31
        cycle=[0.1524]*31
        #at bus occupancy of 20
        bus=[1.42]*31
        #at train occupancy of 146/200
        rail=[0.751]*31
        
        avg=np.array(modal_walk)*np.array(walk)+np.array(modal_cycle)*np.array(cycle) \
            +np.array(modal_bus)*np.array(bus)+np.array(modal_rail)*np.array(rail)

        return avg
        
class Adoption_Rate:
    """
    Calculates adoption rates (percentage of new cars sold of a specific fuel type) from 1989 to 2050
    Takes hisotrical data and estimates furture rates
    """
    def __init__(self,phase_out_date,phase_out_hybrid):
        self.p=phase_out_date
        self.ph=phase_out_hybrid
        
    def adoption_bev(self):
        """
        Adoption of Battery-Electric Cars is the difference between 100% and all other fuel types
        """
        adoption_bev=[]
        adoption_diesel=Adoption_Rate(self.p,self.ph).adoption_diesel()
        adoption_petrol=Adoption_Rate(self.p,self.ph).adoption_petrol()
        adoption_hybrid=Adoption_Rate(self.p,self.ph).adoption_hybrid()
        adoption_plugin=Adoption_Rate(self.p,self.ph).adoption_plugin()
        
        for i in range(0,62):
            adoption_bev.append(100-adoption_diesel[i]-adoption_petrol[i]-adoption_hybrid[i]-adoption_plugin[i])
        
        return(adoption_bev)
    
        
    def adoption_diesel(self):
        """
        Creates list of adoption rates of diesel cars from 1989 to 2050 in percentage
        Assumes linear decrease from historical data
        """
        diesel_increase=Mathematics.poly_fit([2001,2002,2003],adop_car_d[0:3],1)
        diesel_decrease=Mathematics.poly_fit([2017,2018,2019,2020],adop_car_d[16:],1)
        
        #adoption_diesel=[0]*9+diesel_increase(range(1997,2002))+self.d.adop_car_d+diesel_decrease(range(2019,2022))+[0]*79
        
       # d2=diesel_increase(range(1997,2002))
        #print(d2)
        
        #adoption_diesel=[0]*9+self.d.adop_car_d
        #print(adoption_diesel)
        
        adoption_diesel=np.append(np.append(np.append([0]*9,diesel_increase(range(1998,2004))),adop_car_d[3:-4]),np.append(diesel_decrease(range(2017,2023)),[0]*28))
        return(adoption_diesel)

    def adoption_plugin(self):
        """
        Plug-in hybrid phase-out date can only take values 2025,2030,2035,2040
        Consists of a gaussian that peaks and drop back to zero at the phase-out date
        """
        x=np.linspace(2020,self.ph-1,self.ph-2021)
        
        def gaussian(x, mu, sig):
            return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
        
        if self.ph==2025:
            adoption_plugin=np.append(np.append(np.append([0]*12,adop_car_ph),10*gaussian(x,2020+(self.ph-2020)/2,5)),[0]*(2050-self.ph+1))
        elif self.ph==2030:
            adoption_plugin=np.append(np.append(np.append([0]*12,adop_car_ph),15*gaussian(x,2020+(self.ph-2020)/2,4)),[0]*(2050-self.ph+1))
        elif self.ph==2035:
            adoption_plugin=np.append(np.append(np.append([0]*12,adop_car_ph),20*gaussian(x,2020+(self.ph-2020)/2,6)),[0]*(2050-self.ph+1))
            #adoption_plugin=np.append(np.append(np.append([0]*12,adop_car_ph),40*gaussian(x,2020+(self.ph-2020)/2,4.8)),[0]*(2050-self.ph+1))
            #adoption_plugin=np.append(np.append(np.append([0]*12,adop_car_ph),70*gaussian(x,2035,8)),[0]*(2050-self.ph+1))
        elif self.ph==2040:
            adoption_plugin=np.append(np.append(np.append([0]*12,adop_car_ph),25*gaussian(x,2020+(self.ph-2020)/2,7)),[0]*(2050-self.ph+1))
        else:
            adoption_plugin=np.append(np.append(np.append([0]*12,adop_car_ph),15*gaussian(x,2020+(self.ph-2020)/2,5)),[0]*(2050-self.ph+1))
        
        return(adoption_plugin)
    
    def adoption_hybrid(self):
        """
        Hybrid phase-out date can only take values 2025,2030,2035,2040
        Consists of a gaussian that peaks and drop back to zero at the phase-out date
        """
              
        x=np.linspace(2020,self.p,self.p-2021)
        
        def gaussian(x, mu, sig):
            return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
        if self.p==2025:
            adoption_hybrid=np.append(np.append(np.append([0]*12,np.array(adop_car_h)),15*gaussian(x,2020+(self.p-2020)/2,6)),[0]*(2050-self.p+1))
        elif self.p==2030:
            adoption_hybrid=np.append(np.append(np.append([0]*12,np.array(adop_car_h)),20*gaussian(x,2020+(self.p-2020)/2,6)),[0]*(2050-self.p+1))
        elif self.p==2035:
            adoption_hybrid=np.append(np.append(np.append([0]*12,np.array(adop_car_h)),25*gaussian(x,2020+(self.p-2020)/2,7)),[0]*(2050-self.p+1))
        elif self.p==2040:
            adoption_hybrid=np.append(np.append(np.append([0]*12,np.array(adop_car_h)),30*gaussian(x,2020+(self.p-2020)/2,9)),[0]*(2050-self.p+1))
        else:
            adoption_hybrid=np.append(np.append(np.append([0]*12,np.array(adop_car_h)),20*gaussian(x,2020+(self.p-2020)/2,6)),[0]*(2050-self.p+1))
            
        return(adoption_hybrid)
    
    def adoption_petrol(self):
        """
        Assumes 100% adoption rate, until historical data, then linear decrease from 2020 to 0 at phase-out date
        """
        decline=Mathematics.straight_fit(2020,adop_car_p[-1],self.p,0,range(2021,self.p))
        diesel_increase=[100]*3-np.array(Mathematics.poly_fit([2001,2002,2003],adop_car_d[0:3],1)(range(1998,2001)))
        full=np.append(np.append(np.append(np.append([100]*9,diesel_increase),adop_car_p),decline),[0]*(2050-self.p+1))
   
        return(full)

    
class Fuel_Consumption:
    """
    Calculates fuel consumption of petrol, diesel and hybrid cars, and the emissions intensity of the electrical grid
    """
    
    def __init__(self,mass):
        """
        Initialises class with data file
        """
        self.m=mass
        
    def petrol(self):
        
        pre_1997=Mathematics.poly_fit([1997,1998,1999],fuel_car_p[0:3],1)
        
        #post_2020_typ=Mathematics.straight_fit(2020,fuel_car_p[-1],2050,4.5,range(2020,2051))
        #post_2020_suv=Mathematics.straight_fit(2020,fuel_car_p[-1],2050,8.7,range(2020,2051))
        
        post_2020=np.append(Mathematics.straight_fit(2020,fuel_car_p[-1],2025,fuel_car_p[-1]+(self.m-1400)*0.0032,range(2020,2026)),
                            [fuel_car_p[-1]+(self.m-1400)*0.0032]*25)
        
        #fuel_petrol_typ=np.append(pre_1997(range(1989,2000)),np.append(fuel_car_p[3:],post_2020_typ))
        #fuel_petrol_suv=np.append(pre_1997(range(1989,2000)),np.append(fuel_car_p[3:],post_2020_suv))
        
        return np.append(pre_1997(range(1989,2000)),np.append(fuel_car_p[3:],post_2020))
        
    def diesel(self):
        
        pre_1997=Mathematics.poly_fit([1997,1998,1999],fuel_car_d[0:3],1)
        
        #post_2020_typ=Mathematics.straight_fit(2020,fuel_car_d[-1],2050,3.3,range(2020,2051))
        #post_2020_suv=Mathematics.straight_fit(2020,fuel_car_d[-1],2050,6.5,range(2020,2051))
        
        post_2020=np.append(Mathematics.straight_fit(2020,fuel_car_d[-1],2025,fuel_car_d[-1]+(self.m-1400)*0.0028,range(2020,2026)),
                            [fuel_car_d[-1]+(self.m-1400)*0.0028]*25)
        
        #fuel_diesel_typ=np.append(pre_1997(range(1989,2000)),np.append(fuel_car_d[3:],post_2020_typ))
        #fuel_diesel_suv=np.append(pre_1997(range(1989,2000)),np.append(fuel_car_d[3:],post_2020_suv))
        
        return np.append(pre_1997(range(1989,2000)),np.append(fuel_car_d[3:],post_2020))
    
    def hybrid(self):
        
        #hybrid=(0.5*(np.array(Fuel_Consumption().petrol())+np.array(Fuel_Consumption().diesel())))
        
        
        official=[1.66]*62
        realworld=[4.4]*62
        return official,realworld
    
    def bev(self):
        
        #bev=Mathematics.straight_fit(2020,15,2050,10,range(2020,2051))
        
        bev=np.append([18]*31,np.append(Mathematics.straight_fit(2020,18,2025,18+(self.m-1400)*0.01,range(2020,2026)),[18+(self.m-1400)*0.01]*25))
        #bev=[18]*62
        
        return(bev)
    
    def electricity(self):
        
        #in kgco2/kwh
        #projection=Mathematics.straight_fit(2020,elec_emissions[-1],2050,0.029,range(2021,2051))
        projection1=Mathematics.straight_fit(2020,elec_emissions[-1],2025,0.095,range(2021,2026))
        projection2=Mathematics.straight_fit(2025,0.095,2035,0.025,range(2026,2036))
        projection3=Mathematics.straight_fit(2035,0.025,2050,0.01,range(2036,2051))
        genemissions=(np.append(np.append(np.append(elec_emissions,projection1),projection2),projection3))
        
        #lca analysis 2060 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2034,0.154,range(2020,2034))
        projection3=Mathematics.straight_fit(2034,0.154,2048,0.083,range(2034,2048))
        projection4=Mathematics.straight_fit(2048,0.083,2060,0.019,range(2048,2051))
        lcaemissions_2060=(np.append(np.append(np.append(projection1,projection2),projection3),projection4))+0.011
        
        #lca analysis 2055 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2032,0.154,range(2020,2032))
        projection3=Mathematics.straight_fit(2031,0.154,2044,0.083,range(2031,2044))
        projection4=Mathematics.straight_fit(2044,0.083,2055,0.019,range(2044,2051))
        lcaemissions_2055=(np.append(np.append(np.append(projection1,projection2),projection3),projection4))+0.011
        
        #lca analysis 2050 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2030,0.154,range(2020,2030))
        projection3=Mathematics.straight_fit(2030,0.154,2040,0.083,range(2030,2040))
        projection4=Mathematics.straight_fit(2040,0.083,2050,0.019,range(2040,2051))
        lcaemissions_2050=(np.append(np.append(np.append(projection1,projection2),projection3),projection4))+0.011
        
        #lca analysis 2045 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2028,0.154,range(2020,2028))
        projection3=Mathematics.straight_fit(2028,0.154,2036,0.083,range(2028,2036))
        projection4=Mathematics.straight_fit(2036,0.083,2045,0.019,range(2036,2045))
        projection5=Mathematics.straight_fit(2045,0.019,2050,0.019,range(2045,2051))
        lcaemissions_2045=(np.append(np.append(np.append(np.append(projection1,projection2),projection3),projection4),projection5))+0.011
        
        #lca analysis 2040 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2027,0.154,range(2020,2027))
        projection3=Mathematics.straight_fit(2027,0.154,2034,0.083,range(2027,2034))
        projection4=Mathematics.straight_fit(2034,0.083,2040,0.019,range(2034,2040))
        projection5=Mathematics.straight_fit(2040,0.019,2050,0.019,range(2040,2051))
        lcaemissions_2040=(np.append(np.append(np.append(np.append(projection1,projection2),projection3),projection4),projection5))+0.011
        
        #lca analysis 2035 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2025,0.154,range(2020,2025))
        projection3=Mathematics.straight_fit(2025,0.154,2030,0.083,range(2025,2030))
        projection4=Mathematics.straight_fit(2030,0.083,2035,0.019,range(2030,2035))
        projection5=Mathematics.straight_fit(2035,0.019,2050,0.019,range(2030,2051))
        lcaemissions_2035=(np.append(np.append(np.append(np.append(projection1,projection2),projection3),projection4),projection5))+0.011
        
        #lca analysis 2030 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2024,0.154,range(2020,2024))
        projection3=Mathematics.straight_fit(2024,0.154,2027,0.083,range(2024,2027))
        projection4=Mathematics.straight_fit(2027,0.083,2030,0.019,range(2027,2030))
        projection5=Mathematics.straight_fit(2030,0.019,2050,0.019,range(2030,2051))
        lcaemissions_2030=(np.append(np.append(np.append(np.append(projection1,projection2),projection3),projection4),projection5))+0.011
    
        #lca analysis 2025 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.608,2020,0.303,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.303,2022,0.154,range(2020,2022))
        projection3=Mathematics.straight_fit(2022,0.154,2024,0.083,range(2022,2024))
        projection4=Mathematics.straight_fit(2024,0.083,2025,0.019,range(2024,2025))
        projection5=Mathematics.straight_fit(2025,0.019,2050,0.019,range(2025,2051))
        lcaemissions_2025=(np.append(np.append(np.append(np.append(projection1,projection2),projection3),projection4),projection5))+0.011
        
        return genemissions,lcaemissions_2060,lcaemissions_2055,lcaemissions_2050,lcaemissions_2045,lcaemissions_2040,lcaemissions_2035,lcaemissions_2030,lcaemissions_2025
        
    def energy_efficiency(self):
        
        #lca analysis 2050 'net-zero' case
        projection1=Mathematics.straight_fit(2010,0.447,2020,0.630,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,0.630,2030,0.738,range(2020,2030))
        projection3=Mathematics.straight_fit(2030,0.738,2040,0.789,range(2030,2040))
        projection4=Mathematics.straight_fit(2040,0.789,2050,0.835,range(2040,2051))
        energy_efficiency=(np.append(np.append(np.append(projection1,projection2),projection3),projection4))
        
        return energy_efficiency
    
    def eroi(self):
        
        #lca analysis 2050 'net-zero' case
        projection1=Mathematics.straight_fit(2010,27.0,2020,20.9,range(2010,2020))
        projection2=Mathematics.straight_fit(2020,20.9,2030,19.2,range(2020,2030))
        projection3=Mathematics.straight_fit(2030,19.2,2040,19.1,range(2030,2040))
        projection4=Mathematics.straight_fit(2040,19.1,2050,19.0,range(2040,2051))
        eroi=(np.append(np.append(np.append(projection1,projection2),projection3),projection4))
        
        return eroi
        
        
class Distance_Driven:
    """
    Calculates distance driven by average car by dividing total km driven by whole fleet from data with number of fleet in data
    """
    
    def __init__(self,miles_driven):
        """
        Initialises class with data file
        """
        self.md=miles_driven
    
    def Lon(self):
        """
        if self.f==0:
            projection=Mathematics.straight_fit(2020,km_lon[-1],2050,km_lon[-1],range(2020,2052))
        elif self.f==1:
            projection=Mathematics.straight_fit(2020,km_lon[-1],2050,32,range(2020,2052))
        elif self.f==-1:
            projection=Mathematics.straight_fit(2020,km_lon[-1],2050,23,range(2020,2052))
        """
        if self.md==-43:
            preprojection=Mathematics.straight_fit(2020,km_lon[-1],2022,km_lon[-1],range(2020,2022))
            projection=Mathematics.straight_fit(2022,km_lon[-1],2041,km_lon[-1]*(1-0.43),range(2022,2041))
            postprojection=Mathematics.straight_fit(2041,km_lon[-1]*(1-0.43),2050,km_lon[-1]*(1-0.43),range(2041,2052))
        
        elif self.md==-0.1:
            preprojection=Mathematics.straight_fit(2020,km_lon[-1],2022,km_lon[-1],range(2020,2022))
            projection=Mathematics.straight_fit(2022,km_lon[-1],2024,km_lon[-1]*(1-0.81),range(2022,2024))
            postprojection=Mathematics.straight_fit(2024,km_lon[-1]*(1-0.81),2050,km_lon[-1]*(1-0.81),range(2024,2052))
            
        else:
            preprojection=Mathematics.straight_fit(2020,km_lon[-1],2022,km_lon[-1],range(2020,2022))
            projection=Mathematics.straight_fit(2022,km_lon[-1],2027,km_lon[-1]*(1+self.md/100),range(2022,2027))
            postprojection=Mathematics.straight_fit(2027,km_lon[-1]*(1+self.md/100),2050,km_lon[-1]*(1+self.md/100),range(2027,2052))
        return np.append(np.append(np.append(np.array(km_lon),preprojection),projection),postprojection)
    
    def twenty(self):
        
        projection=Mathematics.straight_fit(2020,km_lon[-1],2050,km_lon[-1]*1.2,range(2020,2052))
        return np.append(km_lon,projection)
    
    def age_distribution(self):
        x=np.array(range(0,31))
        y=(x**2-60*x+900)/900
        area_under=np.trapz(y,x)
        return y/area_under
        
    

In [9]:
def initialise_fleet(year,l,p,ph,m,fs,md,c,e):
    """
    Generate a vehicle fleet based on base data. Evolve it through time based on projections.
    e.g. initialise_fleet(2019,2030,1,0.05)
    """
    
    #if model in uk (0) or london (1)
    if l==0:
        car_fleet_size=uk_car_2019
    elif l==1:
        car_fleet_size=lon_car_2019
    adoption_diesel=Adoption_Rate(p,ph).adoption_diesel()
    adoption_bev=Adoption_Rate(p,ph).adoption_bev()
    adoption_plugin=Adoption_Rate(p,ph).adoption_plugin()
    adoption_petrol=Adoption_Rate(p,ph).adoption_petrol()+Adoption_Rate(p,ph).adoption_hybrid()
    
    car_list=[]
    
    #print('the following number of cars were made every year in the 2019 fleet')
    
    
    #cut first 10 years of car age and distribute proportionally on all other years
    new_car_age=list(car_age[i]/sum(car_age[10:]) for i in range(10,31))
    
    #for every year of manufacture i.e. 1999 to 2019 (21 years)
    
    #for every year of manufacture i.e. 1989 to 2019 (31 years)
    for i in range(0,len(new_car_age)):
        
        dcount=0
        bcount=0
        pcount=0
        hcount=0
        
        #make diesel cars (fuel type 0)
        # % of cars that age * 2019 car fleet size * adoption of diesel cars that year /100 from adoption percentage
        for j in range(0,int(new_car_age[i]*car_fleet_size*adoption_diesel[i]/100)):
            c=Vehicle(body_type=0,fuel_type=0,age=(1999+i),mass=m,china=c,elec=e)
            car_list.append(c)
            dcount=dcount+1
        
        #make bev fuel type 4
        for j in range(0,int(new_car_age[i]*car_fleet_size*adoption_bev[i]/100)):
            c=Vehicle(body_type=0,fuel_type=4,age=(1999+i),mass=m,china=c,elec=e)
            car_list.append(c)
            bcount=bcount+1
            
        #make plug-in hybrid fuel type 2
        for j in range(0,int(new_car_age[i]*car_fleet_size*adoption_plugin[i]/100)):
            c=Vehicle(body_type=0,fuel_type=2,age=(1999+i),mass=m,china=c,elec=e)
            car_list.append(c)
            hcount=hcount+1
        
        #make petrol fuel type 1
        for j in range(0,int(new_car_age[i]*car_fleet_size*adoption_petrol[i]/100)):
            c=Vehicle(body_type=0,fuel_type=1,age=(1999+i),mass=m,china=c,elec=e)
            car_list.append(c)
            pcount=pcount+1
            
        #print('year:',1989+i,'petrol cars:', pcount,'diesel cars:', dcount,'bev cars:',bcount,'phev cars:',hcount)
        
    #print('all cars=',len(car_list))
    #print('petrol&hybrid=',sum(p.fuel_type == 1 for p in car_list))
    #print('diesel=',sum(p.fuel_type == 0 for p in car_list))
    #print('bev=',sum(p.fuel_type == 4 for p in car_list))
    #print('plug-in hybrid=',sum(p.fuel_type == 2 for p in car_list))
    
    return car_list


In [1]:
def evolve_fleet(year,l,p,ph,g1,g2,m,fs,md,rf,c,e):
    """
    evolve_fleet(2019,2030,1,0.05)
    """
    
    #create car list from vehicle initialisation class sorted by year of 
    #manufacture in ascending order (oldest cars first)
    car_list=initialise_fleet(year,l,p,ph,m,fs,md,c,e) 
    
    total_cars=[]
    bev_cars=[]
    petrol_cars=[]
    diesel_cars=[]
    plugin_cars=[]
    conv_cars=[]
    avg_emissions=[]
    ages=[]
    demand_difference=[]
    driven_emissions=[]
    electric_emiss=[]
    tailpipe_emiss=[]
    prod_emissions=[]
    ev_prod_emissions=[]
    ice_prod_emissions=[]
    conv_prod_emissions=[]
    ev_prod_energy=[]
    ice_prod_energy=[]
    conv_prod_energy=[]
    elec_demand=[]
    foss_demand=[]
    wind_embedded_emissions=[]
    km_driven=[]
    
    #new cars = fleet size from projection - cars currently in the object list (after removal)
    #slight discrepancy between vehicle fleet sizes list I have constructed and actual data (ie for fleet projection)

    #find future fleet size using user input for % increase from fleet size in 2019 to 2050 levels
    #fleetsize=Mathematics.straight_fit(2019,no_car_lon[-1],2051,no_car_lon[-1]*(1+fs/100),range(2020,2052))
    fleetsize=np.append(Mathematics.straight_fit(2019,no_car_lon[-1],2041,no_car_lon[-1]*(1+fs/100),range(2020,2041)),[no_car_lon[-1]*(1+fs/100)]*11)

    
    #for evolution of fleet from 2020 to 2050
    for i in range(0,31):
        #print('year=',2020+i)
        
        #find age distribution for violin plot
        age_list=[(2020+i-x.age) for x in car_list]
        ages.append(age_list)
        
        before_removal=len(car_list)
        
        #count number of ICE cars removed
        ice_count=0
        
        #remove cars based on a grandfathering policy
        #EVs have default lifetime of 20 years, whereas ICEVs have variable lifetimes set by g1 (cars made prior to 2020) 
        #and g2 (cars made post 2020)
                        
        car_list2=[]
        for j in range(0,len(car_list)):
            if car_list[j].fuel_type==4:
                if car_list[j].age>=2020+i-20:
                    car_list2.append(car_list[j])
            #for retrofitted cars, lifetime is 10 years longer than normal ICE scrapping age (post-2020)
            elif car_list[j].fuel_type==3:
                if car_list[j].age>=2020+i-g2-10:
                        car_list2.append(car_list[j])
            else:
                 
                if car_list[j].age<2020:
                    #for cars made prior to 2020, allow a longer age
                    if car_list[j].age>=2020+i-g1:
                        car_list2.append(car_list[j])
                    else:
                        if car_list[j].fuel_type==0:
                            ice_count=ice_count+1
                        if car_list[j].fuel_type==1:
                            ice_count=ice_count+1
                    
                else:
                    #for cars made post 2020, make scrap policy happen earlier
                    if car_list[j].age>=2020+i-g2:
                        car_list2.append(car_list[j])
                    else:
                        if car_list[j].fuel_type==0:
                            ice_count=ice_count+1
                        if car_list[j].fuel_type==1:
                            ice_count=ice_count+1                   
                    
        car_list=car_list2
        
        after_removal=len(car_list)
        
        #find how many cars removed
        cars_removed=before_removal-after_removal
        
        #find number of new cars needed every year to satisfy total fleet size
        #cars in year n+1 - cars in year n + cars removed
        #number of new cars depending on adoption rates
        #new_cars=fleetsize[i+1]-fleetsize[i]+cars_removed
        new_cars=fleetsize[i+1]-after_removal
        
        #making new cars
        if new_cars>0:
            #1/3 of old fossil fuel cars are retrofitted
            #if there are more new cars than ICEV cars removed, retrofit 1/3rd of ICEV cars 
            if new_cars-ice_count>=0:
                new_conv=int(rf*ice_count)
            #if there are less new cars than ICEV cars removed, retrofit 1/3rd of the number of new cars
            if new_cars-ice_count<0:
                new_conv=int(rf*new_cars)
            
            rest_of_cars=new_cars-new_conv
            new_BEV=int(Adoption_Rate(p,ph).adoption_bev()[i+30]/100*rest_of_cars)
            new_diesel=int(Adoption_Rate(p,ph).adoption_diesel()[i+30]/100*rest_of_cars)
            new_plugin=int(Adoption_Rate(p,ph).adoption_plugin()[i+30]/100*rest_of_cars)
            new_petrol=int((Adoption_Rate(p,ph).adoption_petrol()[i+30]/100+Adoption_Rate(p,ph).adoption_hybrid()[i+30]/100)*rest_of_cars)
            #make cars based on adoption rate* new cars
            for j in range(0,new_BEV):
                car=Vehicle(body_type=0,fuel_type=4,age=2020+i,mass=m,china=c,elec=e)
                car_list.append(car)
            for j in range(0,new_petrol):
                car=Vehicle(body_type=0,fuel_type=1,age=2020+i,mass=m,china=c,elec=e)
                car_list.append(car)
            for j in range(0,new_diesel):
                car=Vehicle(body_type=0,fuel_type=0,age=2020+i,mass=m,china=c,elec=e)
                car_list.append(car)
            for j in range(0,new_plugin):
                car=Vehicle(body_type=0,fuel_type=2,age=2020+i,mass=m,china=c,elec=e)
                car_list.append(car)
            for j in range(0,new_conv):
                car=Vehicle(body_type=0,fuel_type=3,age=2020+i,mass=m,china=c,elec=e)
                car_list.append(car)
            
            
        count=0    
        #if more cars need to be removed due to a modal shift, then remove oldest ICEV cars
        if new_cars<0:
            car_list.sort(key=lambda x: x.age, reverse=False)
            car_list.sort(key=lambda x: x.fuel_type, reverse=False)
            del car_list[0:-int(new_cars)]
            
            """
            j=0
            #delete number of new cars from list (only petrol and diesel)
            while count<-new_cars:
                if car_list[j].fuel_type==1 or car_list[j].fuel_type==2:
                    
                    count=count+1
                j=j+1
            """
            
            
        
        #print('fleetsize[i+1]',fleetsize[i+1],'fleetsize[i]',fleetsize[i],'cars removed from scrapping',cars_removed)
        
        #print('year=',2020+i,'new cars=',new_cars, 'new bevs=', new_BEV, 'new petrol=',new_petrol, 'new diesel', new_diesel, 'new plugin', new_plugin)
        
        #print('cars removed from modal shift',count)
        
            
        #find no of cars
        total_cars.append(len(car_list))
        #print('length of car list',len(car_list))
        bev_cars.append(sum(p.fuel_type == 4 for p in car_list))
        #print(sum(p.fuel_type == 0 for p in car_list))
        petrol_cars.append(sum(p.fuel_type==1 for p in car_list))
        diesel_cars.append(sum(p.fuel_type==0 for p in car_list))
        plugin_cars.append(sum(p.fuel_type==2 for p in car_list))
        conv_cars.append(sum(p.fuel_type==3 for p in car_list))
        
        #find carbon intensity
        total_emissions=sum(p.emissions(2020+i)[0] for p in car_list)
        avg_emissions.append(total_emissions/len(car_list))
        #print('total emissions',total_emissions,'average emissions',avg_emissions[i])
        
    
        demand_diff=Distance_Driven(md).twenty()[i+27]*1000000000-Distance_Driven(md).Lon()[i+27]*1000000000
        if demand_diff>0:
            demand_difference.append(demand_diff)
        if demand_diff<=0:
            demand_difference.append(0)
        
        dist=Distance_Driven(md).Lon()[i+27]*1000000000/len(car_list)
        
        if md==20:
            dist=Distance_Driven(md).twenty()[i+27]*1000000000/len(car_list)
        
        driving_emissions=[]
        electric_emissions=[]
        tailpipe_emissions=[]
        production_emissions=[]
        ice_production_emissions=[]
        ev_production_emissions=[]
        ice_production_energy=[]
        ev_production_energy=[]
        conv_production_emissions=[]
        conv_production_energy=[]
        elec_demand_yearly=[]
        foss_demand_yearly=[]
        kilometres=[]
        #for every car in list
        for j in range(0,len(car_list)):
            #add emissions from that car to yearly list (emissions=carbon intensity*quadratic age distribution*km scale factor)
            kilometres.append(dist)
            #print('KILOMETRES IS',kilometres)
            if car_list[j].fuel_type==4:
                #0.15kwh/km * km
                #energy consumption of EV driving + energy demand for ev charging point infrastructure (1.36MJ/kWh)
                elec_demand_yearly.append(Fuel_Consumption(m).bev()[i+30]/100*dist+Fuel_Consumption(m).bev()[i+30]/100*dist*1.36/3.6)
                #emissions in gCO2e
                #emissions from driving EVs + emissions from EV charging infrastructure 94.06 gCO2e/kWh
                electric_emissions.append(car_list[j].emissions(2020+i)[0]*dist+Fuel_Consumption(m).bev()[i+30]/100*dist*94.06)
            if car_list[j].fuel_type==0:
                #MJ of fossil fuel energy (l/km fuel consumption*energy density of fuel*km driven)
                foss_demand_yearly.append(Fuel_Consumption(m).diesel()[car_list[j].age-1989]/100*36.9*dist)
                tailpipe_emissions.append(car_list[j].emissions(2020+i)[0]*dist)
            if car_list[j].fuel_type==1:
                foss_demand_yearly.append(Fuel_Consumption(m).petrol()[car_list[j].age-1989]/100*33.7*dist)
                tailpipe_emissions.append(car_list[j].emissions(2020+i)[0]*dist)
            if car_list[j].fuel_type==2:
                #if phev, add elec demand from phev
                #utility factor of 39% as found by the ICCT
                elec_demand_yearly.append(0.39*(Fuel_Consumption(m).bev()[i+30]/100*dist+Fuel_Consumption(m).bev()[i+30]/100*dist*1.36/3.6))
                electric_emissions.append(car_list[j].emissions(2020+i)[1]*dist+0.39*Fuel_Consumption(m).bev()[i+30]/100*dist*94.06)
                foss_demand_yearly.append(0.61*Fuel_Consumption(m).petrol()[car_list[j].age-1989]/100*33.7*dist)
                tailpipe_emissions.append(car_list[j].emissions(2020+i)[2]*dist)
            if car_list[j].fuel_type==3:
                elec_demand_yearly.append(Fuel_Consumption(m).bev()[i+30]/100*dist+Fuel_Consumption(m).bev()[i+30]/100*dist*1.36/3.6)
                electric_emissions.append(car_list[j].emissions(2020+i)[0]*dist+Fuel_Consumption(m).bev()[i+30]/100*dist*94.06)

            if car_list[j].age==(2020+i):
                production_emissions.append(car_list[j].prod_emissions(2020+i)*1000)
                if car_list[j].fuel_type==2:
                    ev_production_emissions.append(car_list[j].prod_emissions(2020+i)*1000)
                    ev_production_energy.append(car_list[j].prod_energy()*1000)
                if car_list[j].fuel_type==0:
                    ice_production_emissions.append(car_list[j].prod_emissions(2020+i)*1000)
                    ice_production_energy.append(car_list[j].prod_energy()*1000)
                if car_list[j].fuel_type==1:
                    ice_production_emissions.append(car_list[j].prod_emissions(2020+i)*1000)
                    ice_production_energy.append(car_list[j].prod_energy()*1000)
                if car_list[j].fuel_type==4:
                    ev_production_emissions.append(car_list[j].prod_emissions(2020+i)*1000)
                    ev_production_energy.append(car_list[j].prod_energy()*1000)
                if car_list[j].fuel_type==3:
                    conv_production_emissions.append(car_list[j].prod_emissions(2020+i)*1000)
                    conv_production_energy.append(car_list[j].prod_energy()*1000)
        
        
        
                
        driven_emissions.append(sum(driving_emissions))
        km_driven.append(sum(kilometres))
        electric_emiss.append(sum(electric_emissions))
        tailpipe_emiss.append(sum(tailpipe_emissions))
        prod_emissions.append(sum(production_emissions))
        ev_prod_emissions.append(sum(ev_production_emissions))
        ice_prod_emissions.append(sum(ice_production_emissions))
        conv_prod_emissions.append(sum(conv_production_emissions))
        ev_prod_energy.append(sum(ev_production_energy))
        ice_prod_energy.append(sum(ice_production_energy))
        conv_prod_energy.append(sum(conv_production_energy))
        #elec demand in kwh
        elec_demand.append(sum(elec_demand_yearly))
        #fossil demand in MJ
        foss_demand.append(sum(foss_demand_yearly))
        #print('driven emissions=',driven_emissions)
        #print('production emissions=',prod_emissions)
        #print('electricity demand=', elec_demand)
    
    avg_energy_efficiency=Fuel_Consumption.energy_efficiency()
    avg_eroi=Fuel_Consumption.eroi()
    #energy demand = electricity demand divided by the overall efficiency of the grid, plus embedded energy over EROI
    energy_demand=np.array(elec_demand)*np.array(avg_energy_efficiency)+(np.array(elec_demand)*np.array(energy_efficiency))/avg_eroi
        
    co2intensity=ModalShare.co2intensity(e)
    mod_energy=ModalShare.energy()
    #average occupancy of car is 1.6, so calculate modal shift emissions for every passenger 
    mod_shift_emiss=np.array(demand_difference)*1.6*np.array(co2intensity)/1000000000
    cum_mod_shift_emiss=np.append(0,np.cumsum(np.array(demand_difference)*1.6*np.array(co2intensity)/1000000000))
    mod_shift_energy=np.array(demand_difference)*1.6*np.array(mod_energy)    
  
    return(total_cars,bev_cars,petrol_cars,diesel_cars,plugin_cars,conv_cars,avg_emissions,ages,driven_emissions,electric_emiss,tailpipe_emiss,prod_emissions,ev_prod_emissions,ice_prod_emissions,conv_prod_emissions,ev_prod_energy,ice_prod_energy,conv_prod_energy,km_driven,elec_demand,foss_demand,demand_difference,mod_shift_emiss,cum_mod_shift_emiss,mod_shift_energy)